# Newark Notebook

In [17]:
import pandas as pd 
import numpy as np
import helpers
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from datetime import timedelta
geolocator = Nominatim(user_agent="myGeolocator", timeout=2)

# read the csv into a dataframe
# only import the columns we will use, so avoid spending time dropping 90 columns later
"""df = pd.read_csv(('0714monm135536.csv'), usecols=['propertyLocation', 'ownersName', 'ownersMailingAddress', 'cityStateZip'])"""

# after this initial instance, we'll read from the pickled dataframes
df = pd.read_pickle('newark.pkl')
errors_df = pd.read_pickle('errors_df.pkl')

# create a column for the owner's full mailing address
"""df['ownersFullMailingAddress'] = df['ownersMailingAddress'] + ', ' + df.cityStateZip"""
# create a column for the property's full address
"""df['propertyFullAddress'] = df.propertyLocation + ' Newark, NJ'"""

'''# returns an array of properties associated with the owne
def getPropertiesOwned(owner=str):
    return df[df.ownersName == owner]['propertyLocation'].unique().tolist()

# number of times property appears in first column
def getUnits(address=str):
    return len(df[df.propertyLocation == address])

# number of the properties owned by the given owner
def getNumberOwned(owner=str):
    return df[(df.ownersName == owner)]['propertyLocation'].count()'''

"# returns an array of properties associated with the owne\ndef getPropertiesOwned(owner=str):\n    return df[df.ownersName == owner]['propertyLocation'].unique().tolist()\n\n# number of times property appears in first column\ndef getUnits(address=str):\n    return len(df[df.propertyLocation == address])\n\n# number of the properties owned by the given owner\ndef getNumberOwned(owner=str):\n    return df[(df.ownersName == owner)]['propertyLocation'].count()"

In [2]:
jc_df = pd.read_pickle('/Users/kylereaves/Desktop/landlord_data/JerseyCity/jersey_city.pkl')

In [7]:
getNumberOwned('ALS NEWARK LLC,')

3

In [16]:
df.rename(columns={"Lat": "latitude", "Long": "longitude"}, inplace=True)

In [18]:
column_order = ['propertyLocation',
                 'ownersName',
                 'ownersMailingAddress',
                 'cityStateZip',
                 'units',
                 'propertiesOwned',
                 'numberPropertiesOwned',
                 'propertyFullAddress',
                 'gCode',
                 'latitude',
                 'longitude']


In [23]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,units,propertiesOwned,numberPropertiesOwned,propertyFullAddress,gCode,latitude,longitude
0,1050-1052 MCCARTER HWY,SEABRA BROTHERS II,260CHESTNUT ST. 2ND FLOOR,"NEWARK, NJ 07105",1,[1050-1052 MCCARTER HWY],1,"1050-1052 MCCARTER HWY Newark, NJ","(1050, McCarter Highway, Newark, Essex County,...",40.743957,-74.167420
1,1155-1157 MCCARTER HWY,LEG 450 BROAD STREET LLC,283-299 MARKET ST,"NEWARK, NJ 07102",1,"[1155-1157 MCCARTER HWY, 2-28 DIVISION ST, 30-...",4,"1155-1157 MCCARTER HWY Newark, NJ","(McCarter Highway, Newark, Essex County, New J...",40.713282,-74.183981
2,23 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NJ 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"23 BRIDGE ST Newark, NJ","(23, Bridge Street, Newark, Essex County, New ...",40.744495,-74.169458
3,25 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NY 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"25 BRIDGE ST Newark, NJ","(25, Bridge Street, Newark, Essex County, New ...",40.744497,-74.169442
4,27 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NJ 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"27 BRIDGE ST Newark, NJ","(27, Bridge Street, Newark, Essex County, New ...",40.744500,-74.169426
...,...,...,...,...,...,...,...,...,...,...,...
24422,878 S 18TH ST,"OJUTALAYO, OLUKUNLE",878-880 SO 18TH ST,"NEWARK, NJ 07108",1,[878-880 S 18TH],1,"878 S 18TH ST Newark, NJ","(878, South 18th Street, Newark, Essex County,...",40.725522,-74.214026
24444,881 S 18TH ST,"HANKINS, JERRY",883 S 18TH ST,"NEWARK, NJ 07108",1,[881-883 S 18TH],1,"881 S 18TH ST Newark, NJ","(881, South 18th Street, Newark, Essex County,...",40.725471,-74.214270
21783,752 S 15TH ST ST,"WHITE,JAMES R & REBA",752 S 15TH ST,"NEWARK, NJ 07103",1,[752 S FIFTEETH ST],1,"752 S 15TH ST ST Newark, NJ","(752, South 15th Street, Newark, Essex County,...",40.728387,-74.210011
18278,243 2ND AVE,WILSON LEON,243 2ND AVE W,"NEWARK, NJ 07107",1,[243 2ND AVE WEST],1,"243 2ND AVE Newark, NJ","(243, 2nd Avenue, Newark, Essex County, New Je...",40.765368,-74.181202


In [19]:
df.columns.to_list() == jc_df.columns.to_list()

False

## check to see if Essex is not included in the geocoded addresses

In [3]:
#df.to_csv('newark.csv', index=None)

In [3]:
df[df.gCode.str.contains('Essex') == False]

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,units,propertiesOwned,numberPropertiesOwned,propertyFullAddress,gCode


In [4]:
errors_df[errors_df.gCode.str.contains('Essex') == False]

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,units,propertiesOwned,numberPropertiesOwned,propertyFullAddress,gCode


In [5]:
df['Lat'] = [g.latitude for g in df.gCode]
df['Long'] = [g.longitude for g in df.gCode]

## check the shape of the errors_df

In [7]:
errors_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 2957 to 34686
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   propertyLocation       36 non-null     object
 1   ownersName             36 non-null     object
 2   ownersMailingAddress   36 non-null     object
 3   cityStateZip           36 non-null     object
 4   units                  36 non-null     int64 
 5   propertiesOwned        36 non-null     object
 6   numberPropertiesOwned  36 non-null     int64 
 7   propertyFullAddress    36 non-null     object
 8   gCode                  0 non-null      object
dtypes: int64(2), object(7)
memory usage: 2.8+ KB


In [86]:
errors_df.propertyLocation.unique(), errors_df.info()

array(['520 16TH ST', '81 C0SSIO DR', '252 W00DSIDE AVE',
       '100 FORREST HILL PK', '71 FORREST HILL CONDO',
       '77 FORREST HILL CONDO', '47 BRANCH BROOK NY', '72 COLOMBIA ST',
       '233 NO 4TH ST', '96 CLIFORD ST', 'VARNUM ST', '112 ELEVENTH AVE',
       '241 18TH ST ST', '110 S 15TH ST S', '355 12TH ST', '8 DOLBY PL',
       '764 SO 18TH ST', '510 IRVINE TURNER BL', '514 IRVINE TURNER BL',
       '181 SEYMOUR ST', '17 CLNTON PL', '55 HOMESTEAD PARK',
       '1 HOMESTEAD PARK', '42 HOMESTEAD PARK', '1 SARAH VAUGHAN PL',
       '167 MAPE AVE.', '34 BOYLAND ST', '310 SANFORD ST', '105 HYATT LN',
       '161 HYATT LN', '121 AVENUE I REAR', '386 RT 1 & 9',
       '362 RT 1 & 9', '106 RT 1 & 9', '314 RT 1 & 9'], dtype=object)

In [1]:
#errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MCWHORTER', 'MC WHORTER')
#errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ML KING', 'DR MARTIN LUTHER KING')
#errors_df.propertyLocation = errors_df.propertyLocation.str.replace('DR MARTIN LUTHER KING', 'DR MARTIN LUTHER KING JR')

# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SCOFIELD', 'SCHOFIELD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('STECHER', 'STETCHER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PARK VIEW', 'PARKVIEW')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SANDFORD', 'SANFORD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('VAL SUMO', 'VALSUMO')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PATTERSON', 'PATERSON')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE E', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('13COLGATE', '13 COLGATE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MOUNTAIN VIEW', 'MOUNTAINVIEW')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('DE GRAW', 'DEGRAW')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('GOTTHART', 'GOTTHARDT')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE W', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ST AR', 'ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('BLVD AR', 'BLVD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE AR', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('NJRR', 'New Jersey Railroad')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HWY AR', 'HWY')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PL AR', 'PL')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('LN AR', 'LN')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HOMESTEAD PK RD', 'HOMESTEAD PARK RD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('13TH R', '13TH')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HWY R', 'HWY')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE R', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ST R', 'ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE,', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE  AR', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ELVENTH', 'ELEVENTH')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('THRITEENTH R', '13TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PKWAY', 'PKWY')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'-\d{1,4}', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('STEAR', 'ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVEEAR', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('BLVDREAR', 'BLVD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('BLVD R', 'BLVD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE  E', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE ,E', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HALSTED', 'HALSTEAD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('BLVDEAR', 'BLVD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HWYEAR', 'HWY')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(', REAR', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PL REAR', 'PL')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MNCHESTER', 'MANCHESTER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MANCHESTRER', 'MANCHESTER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('STUYVSNT', 'STUYVESANT')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SEVEN10TH', '17TH')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SEVNTEENTH', '17TH')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('BLVDE', 'BLVD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ELLIOT', 'ELLIOTT')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HENNESY', 'HENNESSEY')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('P REAR', 'P')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FOREST', 'FORREST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'\\d{0,2}TH', r'\\d{0,2}TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PARK ROA', 'PARK')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('WHEELR', 'WHEELER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HORATIOST', 'HORATIO ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PKW', 'PKWY')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('M L KING BLVD', 'DR MARTIN LUTHER KING JR BLVD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('BLDG551', '551')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('NORFLOK', 'NORFOLK')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIFTEETH', 'FIFTEENTH')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ST H1', 'ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('VANDERPOL', 'VANDERPOOL')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HAZELLWOOD', 'HAZELWOOD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MILAND', 'MIDLAND')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ORNAGE', 'ORANGE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('LN REAR', 'LN')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FRELINGHYSEN', 'FRELINGHUYSEN')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE  REAR', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('12TH', '12TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('13TH', '13TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('14TH', '14TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('15TH', '15TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('16TH', '16TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('17TH', '17TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('18TH', '18TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('19TH', '19TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('20TH', '20TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ORTH 12TH ST ST	', '12TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('18TH ST ST', '18TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVEEST', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIFTEENTH', '15TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ORTH 12TH ST ST', '12TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVEEST', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('S 16TH ST S', '16TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('NO 4TH ST', ' 4TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('18TH ST ST', ' 18TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('S 15TH ST S	', '15TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('S 15TH ST S', '15TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SO 18TH ST', '18TH ST')

errors_df['propertyFullAddress'] = errors_df.propertyLocation + ' Newark, NJ'

NameError: name 'errors_df' is not defined

In [88]:
errors_df['gCode'] = errors_df.propertyFullAddress.apply(geolocator.geocode)

In [89]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [90]:
errors_df.iloc[geocode_errors]

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,units,propertiesOwned,numberPropertiesOwned,propertyFullAddress,gCode
2957,520 16TH ST,"ASCENSION CAPITAL PARTNERS I,LLC","59 LINCOLN PARK,#200","NEWARK, NJ 07102",1,[520-526 S 16TH S],1,"520 16TH ST Newark, NJ",None
3688,81 C0SSIO DR,"JACKSON, DWAYNE & ANNA",81 COSSIO DR,"NEWARK, NJ 07103",1,[81 C0SSIO DR],1,"81 C0SSIO DR Newark, NJ",None
8697,252 W00DSIDE AVE,"RIVERA, GABRIEL & DEBORAH",252 WOODSIDE AVE,"NEWARK, NJ 07104",1,"[252 W00DSIDE AVE,]",1,"252 W00DSIDE AVE Newark, NJ",None
8927,100 FORREST HILL PK,FOREST HILLS 1 LLC ET AL,110 CHESTNUT RIDGE RD,"MONTVALE,NJ 07645",1,[100-102 FOREST HILL PK],1,"100 FORREST HILL PK Newark, NJ",None
9041,71 FORREST HILL CONDO,"FOREST HILLS 1, LLC, ET AL",110 CHESTNUT RIDGE RD 301,"MONTVALE, NJ 07645",2,"[84 FOREST HILL PKWY, 88 FOREST HILL PKWY, 96 ...",418,"71 FORREST HILL CONDO Newark, NJ",None
9044,71 FORREST HILL CONDO,"FOREST HILLS 1, LLC, ET AL",110 CHESTNUT RIDGE RD 301,"MONTVALE, NJ 07645",2,"[84 FOREST HILL PKWY, 88 FOREST HILL PKWY, 96 ...",418,"71 FORREST HILL CONDO Newark, NJ",None
9054,77 FORREST HILL CONDO,"FOREST HILLS 1, LLC, ET AL",110 CHESTNUT RIDGE RD 301,"MONTVALE, NJ 07645",1,"[84 FOREST HILL PKWY, 88 FOREST HILL PKWY, 96 ...",418,"77 FORREST HILL CONDO Newark, NJ",None
10152,47 BRANCH BROOK NY,"FOREST HILLS 1, LLC, ET AL",110 CHESTNUT RIDGE RD 301,"MONTVALE, NJ 07645",1,"[84 FOREST HILL PKWY, 88 FOREST HILL PKWY, 96 ...",418,"47 BRANCH BROOK NY Newark, NJ",None
11157,72 COLOMBIA ST,"GIAMMONA, PETER",7 JESSICA WAY,"WAYNE, NJ 07470",1,[72 COLOMBIA ST],1,"72 COLOMBIA ST Newark, NJ",None
12340,96 CLIFORD ST,FERREIRA CONSTANTINO & MARIA,96 CLIFFORD ST,"NEWARK, NJ 07105",1,[96 CLIFORD ST],1,"96 CLIFORD ST Newark, NJ",None


## append nonNA values to the dataframe before pickling it
## write over errors_df
## pickle errors_df

In [91]:
#non NA values appened to the dataframe and then pickled
df.append(errors_df.dropna(axis=0, subset=['gCode'])).to_pickle('newark.pkl')
# writing over the old errors_df
errors_df = errors_df.iloc[geocode_errors].copy()
# saving the new errors_df
errors_df.to_pickle('errors_df.pkl')

In [79]:
first_cleaning = timedelta(seconds=17439)
second_cleaning = timedelta(seconds=819.5)